In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import geojson
import plotly.express as px
import plotly.io as pio
import json

In [2]:
def load_trips(list):
    df = pd.DataFrame()
    for month in list:
        print(month)
        trips = pq.read_table(f'data/yellow_tripdata_2022-{month:02}.parquet')
        trips_df = trips.to_pandas()
        # Append data
        df = pd.concat((df, trips_df), ignore_index=True)# df.concat(trips_df, ignore_index=True)

    return df
list = range(1,13)
df = load_trips(list)
#df.to_csv('data/yellow_tripdata_combined.csv')
#df = pd.read_csv('data/yellow_tripdata_combined.csv')
print(df.shape)

1
2
3
4
5
6
7
8
9
10
11
12
(39656098, 19)


In [3]:
df["pickup_at"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["dropoff_at"] = pd.to_datetime(df["tpep_dropoff_datetime"])

In [4]:
df["duration"] = (df["dropoff_at"] - df["pickup_at"]).dt.total_seconds()
df["tip_percentage"] = 100*df["tip_amount"] / df["total_amount"]

In [5]:
print(df.columns)

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'pickup_at',
       'dropoff_at', 'duration', 'tip_percentage'],
      dtype='object')


In [7]:
df = df.dropna()
print(df.shape)

(38281537, 23)


In [ ]:
def remove_outliers(df, attribute, min = None, max = None, include_min = True, include_max = True):
    if min is not None:
        if include_min:
            df = df[df[attribute]>=min]
        else:
            df = df[df[attribute]>min]
    if max is not None:
        if include_max:
            df = df[df[attribute]<=max]
        else:
            df = df[df[attribute]<max]
    return df

In [ ]:
df = remove_outliers(df, "duration", min = 60, max = 10800)

In [ ]:
df = remove_outliers(df, "passenger_count", max = 6)

In [ ]:
df = remove_outliers(df, "trip_distance", min = 0, max = 100, include_min = False)

In [ ]:
df = remove_outliers(df, "RatecodeID", min = 1, max = 6)

In [ ]:
df = remove_outliers(df, "payment_type", min = 1, max = 2)

In [ ]:
df = remove_outliers(df, "fare_amount", min = 2.5, max = 250, include_min = False)

In [ ]:
df = remove_outliers(df, "extra", min = 0)

In [ ]:
df = remove_outliers(df, "mta_tax", min = 0)

In [ ]:
df = remove_outliers(df, "tip_percentage", min = 0, max = 50)

In [ ]:
df = remove_outliers(df, "tolls_amount", min = 0, max = 100)

In [ ]:
df = remove_outliers(df, "improvement_surcharge", min = 0)

In [ ]:
df = remove_outliers(df, "total_amount", min = 2.5, max = 1000, include_min=False)

In [ ]:
df = remove_outliers(df, "congestion_surcharge", min = 0)

In [ ]:
df = remove_outliers(df, "airport_fee", min = 0)

In [ ]:
df.shape

(37067652, 22)

In [ ]:
df.to_csv('data/yellow_tripdata_combined_cleaned.csv')

In [ ]:
focus = ['trip_distance','PULocationID', 'DOLocationID','fare_amount',
         'tip_amount','total_amount','pickup_at', 'dropoff_at','duration']
df_focus = df[focus]

df_focus.to_csv('data/yellow_tripdata_combined_cleaned_focus.csv')